**Multi-Layer-Perceptrons**

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
dataset = pd.read_csv('Dataset.csv')

In [48]:
cols = dataset.columns

unnamed_cols = [col for col in cols if col.startswith('Unnamed')]

dataset = dataset.drop(columns=unnamed_cols)

dataset.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,668,33.0,3,0.00,2,1,0,181449.97,0
1,627,33.0,1,0.00,2,1,1,49503.50,0
2,678,40.0,10,0.00,2,1,0,184866.69,0
3,581,34.0,2,148882.54,1,1,1,84560.88,0
4,716,33.0,5,0.00,2,1,1,15068.83,0


In [49]:
input_data = dataset.iloc[:, :-1]
output_data = dataset.iloc[:, -1]

In [50]:
from sklearn.preprocessing import StandardScaler

In [51]:
ss = StandardScaler()
input_data = pd.DataFrame(ss.fit_transform(input_data), columns=input_data.columns)

In [52]:
input_data.shape

(175028, 8)

In [53]:
import tensorflow
from keras.callbacks import EarlyStopping
from keras.regularizers import L2
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout

In [54]:
ann = Sequential()

In [55]:
ann.add(Dense(6,input_dim=8, activation='relu', kernel_regularizer=L2(0.01)))
ann.add(BatchNormalization(),Dropout(0.2))
ann.add(Dense(5, activation='relu'))
ann.add(BatchNormalization(),Dropout(0.2))
ann.add(BatchNormalization(),Dropout(0.2))
ann.add(Dense(4, activation='relu'))
ann.add(BatchNormalization(),Dropout(0.2))
ann.add(BatchNormalization(),Dropout(0.2))
ann.add(Dense(3, activation='relu'))
ann.add(BatchNormalization(),Dropout(0.2))
ann.add(Dense(2, activation='relu'))
ann.add(BatchNormalization(),Dropout(0.2))
ann.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [56]:
ann.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.2, random_state=42)
x_train.shape

(140022, 8)

In [58]:
x_test

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
55707,0.774966,-0.576544,-1.430010,-0.899885,-1.007031,-1.737531,1.002494,-1.148306
92588,1.502032,0.649829,1.771530,-0.899885,-1.007031,0.575529,-0.997512,-0.669072
33131,1.945665,0.649829,-0.718557,1.110575,-1.007031,-1.737531,1.002494,0.702347
49808,-0.740783,0.426852,1.415803,-0.899885,0.813766,-1.737531,1.002494,0.218948
48649,0.417594,0.984294,-1.074283,-0.899885,-1.007031,0.575529,-0.997512,0.777575
...,...,...,...,...,...,...,...,...
44433,-0.925630,-1.579940,0.348623,-0.899885,0.813766,0.575529,1.002494,0.817414
161,0.725673,0.872805,-1.074283,-0.899885,-1.007031,0.575529,-0.997512,0.090681
14415,-0.186241,-0.799521,-1.074283,1.124120,-1.007031,-1.737531,-0.997512,-0.767654
89140,-0.124625,0.203875,0.704350,1.245092,-1.007031,0.575529,-0.997512,0.472539


In [69]:
ann.fit(x_train, y_train, epochs=20,batch_size=50, validation_data=(x_test, y_test),callbacks=[EarlyStopping(monitor='val_accuracy')])

Epoch 1/20
2801/2801 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - accuracy: 0.8492 - loss: 0.3552 - val_accuracy: 0.8530 - val_loss: 0.3424
Epoch 2/20
2801/2801 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.8497 - loss: 0.3531 - val_accuracy: 0.8523 - val_loss: 0.3447


In [71]:
train_accuracy = ann.history.history['accuracy']
test_accuracy = ann.history.history['val_accuracy']

In [72]:
# plt.plot([i for i in range(1,8)],train_accuracy )
# plt.plot([i for i in range(1,8)],test_accuracy , c='red')
# plt.show()

In [73]:
prd1=ann.predict(x_train)
prd_data1 = []
for i in prd1:
  if i>0.5:
    prd_data1.append(1)
  else:
    prd_data1.append(0)

4376/4376 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step


In [74]:
prd=ann.predict(x_test)

1094/1094 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [75]:
prd_data = []
for i in prd:
  if i>0.5:
    prd_data.append(1)
  else:
    prd_data.append(0)

In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prd_data)*100

85.22824658629949

In [77]:
accuracy_score(y_train, prd_data1)*100

85.24588993158217

In [78]:
prd=ann.predict(np.array([[0.725673,	0.872805,	-1.074283,	-0.899885,	-1.007031	,0.575529	,-0.997512	,0.090681]])) # Convert the input list to a NumPy array
prd_data = []
for i in prd:
  if i>0.5:
    prd_data.append(1)
  else:
    prd_data.append(0)
prd_data

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


[1]

In [68]:
y_test

,Exited
55707,0
92588,0
33131,0
49808,0
48649,0
...,...
44433,0
161,1
14415,0
89140,0
